In [ ]:
# 1. `.unwrap()` là gì?

**Trong RTK Query**:
    - Khi gọi một mutation (Ví dụ: `createProduct`)
    - Hàm sẽ trả về một Promise
    - Promise này LUÔN LUÔN `fulfilled` (thành công) về mặt kỹ thuật, ngay cả khi API trả về lỗi 4xx hoặc 5xx

> Kết quả trả về mặc định là một object Action của Redux có dạng:
    - Thành công: `{ data: {id: 1, name: 'A'}, meta: ... }`
    - Thất bại: `{ error: {status: 500, data: 'Lỗi server'}, meta: ... }`

> `.unwrap()` là một hàm tiện ích giúp "Bóc tách" cái vỏ bọc object Action của Redux ở trên ra để trả về:
    - Dữ liệu thô (raw data) nếu thành công: Tức là trả thẳng ra `data`
    - Ném ra lỗi (throw error) nếu thất bại (để có thể bắt lỗi ở `catch`)

In [ ]:
## Tưởng tượng:
- Hãy tưởng tượng bạn nhờ RTK Query đi lấy một món đồ từ Server về:

## TH1: Không dùng `.unwrap()` (Mặc định)
RTK Query giống như một người giao hàng "an toàn tuyệt đối"
  - DÙ món đồ bên trong là **Iphone 15** (thành công) hay **cục gạch** (lỗi server), người giao hàng luôn **đóng gói nó cẩn thận vào một CÁI HỘP** rồi đưa cho bạn
  - Việc nhận cái hộp luôn được coi là **"Thành công"** (Promise fulfilled)
  - **Hệ quả**: Bạn nhận cái hộp, bạn phải tự tay mở ra, nhìn vào trong mới biết là Iphone hay cục gạch
    - Code: `if (hop.cucGach) { ... } else { ... }`

## TH2: Có dùng `.unwrap()` (Bóc vỏ)
- Đúng như tên gọi (Unwrap = Bóc vỏ/Mở hộp). Hàm này hoạt động như một **"Bộ lọc kiểm tra"** đứng chắn ngay cửa.
- Khi người giao hàng mang cái hộp về, `.unwrap()` sẽ giật lấy cái hộp và **xé toạc vỏ hộp ra** ngay lập tức.

**Kịch bản A**:
  - Nếu bên trong là **Iphone**, nó đưa trần món đồ (chiếc Iphone) cho bạn dùng luôn (Promise Resolve với `data`)

**Kịch bản B**:
  - Nếu bên trong là **cục gạch** (Lỗi), nó thấy "ngứa mắt", và **ném cục gạch xuống đất** (Throw Error) ngay lập tức để báo động

> Hệ quả: Bạn không cần mở hộp nữa. 
  + Nếu món đồ đến tay bạn, chắn chắn nó là Iphone.
  + Nếu là gạch, tiếng động "xoảng" (Catch) đã vang lên rồi.

In [ ]:
# `.unwrap()` giải quyết nỗi đau nào?

## Nỗi đau:
- Không thể dùng Try/Catch theo cách thông thường

  + Nếu không có `.unwrap()`, khi bạn dùng `await` cho một mutation, code sẽ không bao giờ nhảy vào block `catch`, kể cả khi API lỗi.
  + Phải viết code kiểm tra điều kiện thủ công và rườm rà

❌ Khi không dùng `.unwrap()`


In [ ]:
const handleCreate = async () => {
    // Gọi API
    const result = await createProduct(formData); 
    
    // Dù API lỗi 500, dòng code này vẫn chạy, không nhảy xuống catch!
    
    // Bạn phải tự check thủ công xem có key 'error' không
    if (result.error) {
        // Xử lý lỗi: toast.error("Có lỗi rồi")
        console.log(result.error);
    } else {
        // Xử lý thành công: toast.success("Tạo xong")
        console.log(result.data);
    }
};

/* 
Vấn đề: 
- Code trở nên dài dòng, khó đọc
- Đi ngược lại thói quen dùng try/catch để bắt lỗi luồng bất đồng bộ
 */

✅ Khi dùng `.unwrap()`

In [ ]:
const handleCreate = async () => {
    try {
        // Nếu API thành công: Trả về thẳng data (không còn vỏ bọc {data: ...})
        const payload = await createProduct(formData).unwrap();
        
        // Code chạy tới đây nghĩa là chắc chắn thành công
        toast.success("Tạo sản phẩm thành công!");
        console.log(payload); // Dữ liệu thật luôn
        
    } catch (error) {
        // Nếu API lỗi: Tự động nhảy vào đây
        toast.error("Toang rồi: " + error.message);
    }
};

In [ ]:
## Khi nào nên dùng `.unwrap()`

- Dùng khi xử lý sự kiện (xử lý sự kiện `click`, `submit form`)
- Khi cần thực hiện hành động TIẾP THEO dựa trên kết quả API:
- VD: Chuyển trang sau khi Logic thành công, hoặc hiện thông báo Toast

In [ ]:
// Ví dụ trong Form Logic

const [login] = useLoginMutation();

const onSubmit = async (data) => {
  try {
    await login(data).unwrap();
    router.push('/dashboard'); // Chỉ chuyển trang nếu login thành công
  } catch (err) {
    setError('password', { message: 'Sai mật khẩu rồi!' }); // Hiện lỗi nếu thất bại
  }
}

# Kịch bản thực tế

In [ ]:
Kịch bản: Bạn có tính năng "Tạo sản phẩm mới". Quy trình thực hiện như sau:
  1. Gọi API tạo sản phẩm
  2. Nếu thành công: 
    + Hiển thị thông báo "Thành công"
    + Đóng form popup
    + Chuyển hướng sang trang chi tiết đó
  3. Nếu thất bại:
    + Hiển thị thông báo lỗi từ server (VD: 'Tên sản phẩm bị trùng')
    + Giữ nguyên form để người dùng chỉnh sửa

## 1. Cách cũ. Không dùng `.unwrap()` (Khá cồng kềnh)
- Khi không dùng `.unwrap`, RTK Query trả về một **Object kết quả (Action)**
- Dù API trả về lỗi 400/500, hàm `await` vẫn coi là code chạy xong xuôi (fulfilled)
> Nó không bao giờ nhảy vào `catch`

In [ ]:
const handleCreateProduct = async (formData) => {
  // 1. Gọi Mutation
  const result = await createProduct(formData);

  // --- NỖI ĐAU BẮT ĐẦU TẠI ĐÂY ---
  
  // Bạn phải tự viết logic "if-else" để kiểm tra xem bên trong có lỗi không
  if (result.error) {
    // 2. Xử lý Lỗi
    // Phải mò mẫm vào result.error để lấy message
    const message = result.error.data?.message || "Lỗi không xác định";
    toast.error(message);
    // Code dừng tại đây, không làm gì thêm
    return; 
  }

  // 3. Xử lý Thành công
  // Muốn lấy data thật, phải chọc vào result.data
  const newProduct = result.data;
  
  toast.success("Tạo thành công!");
  setOpenModal(false); // Đóng modal
  navigate(`/products/${newProduct.id}`); // Chuyển trang
};

In [ ]:
Nhược điểm:
  - Code lộn xộn: Trộn lẫn logic kiểm tra (if/else) với logic nghiệp vụ
  - Dễ quên: Nếu quên check `if (result.error)`, code bên dưới vẫn sẽ chạy và gây lỗi `undefined` khi cố truy cập `result.data`
  - Khó kết hợp: Nếu bạn cần làm 2 việc liên tiếp (Tạo sản phẩm -> Sau đó upload ảnh cho sản phẩm đó), rơi vào cảnh "if lồng if"

## 2. Cách mới: CÓ dùng `.unwrap()` (Sạch sẽ, Chuẩn bài)
- Khi dùng `.unwrap()`, 
    + Biến nó thành một Promise chuẩn. 
    + Nếu API lỗi, nó sẽ ném (throw) lỗi ngay lập tức
    + Giúp dùng cấu trúc `try/catch` quen thuộc

In [ ]:
const handleCreateProduct = async (formData) => {
  try {
    // 1. Gọi Mutation và Bóc tách (Unwrap) ngay lập tức
    // Nếu API lỗi, nó "nhảy" thẳng xuống catch, không chạy dòng dưới nữa.
    const newProduct = await createProduct(formData).unwrap();

    // 2. Xử lý Thành công (Code chạy đến đây nghĩa là chắc chắn 100% thành công)
    // newProduct ở đây là data sạch, không cần chấm .data nữa
    toast.success("Tạo thành công!");
    setOpenModal(false);
    navigate(`/products/${newProduct.id}`);

  } catch (error) {
    // 3. Xử lý Lỗi (Tất cả lỗi gom về một mối)
    console.log("Lỗi nè:", error);
    const message = error.data?.message || "Toang rồi ông giáo ạ!";
    toast.error(message);
  }
};

In [ ]:
Ưu điểm:
  - Luồng code thẳng: Đọc code từ trên xuống dưới là luồng thành công. Lỗi được gom riêng ra một chỗ.
  - Early Exit: Nếu dòng `unwrap()` bị lỗi, các dòng code logic phía sau (đóng modal, chuyển trang) tự động không chạy. Không cần viết `return` thủ công
  - Đồng bộ với thói quen: Chúng ta quen dùng `try/catch` cho `async/await`

# VÍ DỤ THẤU HIỂU NỖI ĐAU:
Tại sao `.unwrap()` là cứu tinh khi chạy chuỗi hành động?

- Giả sử bạn phải làm 2 bước:
    1. Tạo Product (lấy ID)
    2. Upload ảnh cho ID đó

In [ ]:
// Không có ".unwrap()"

/* Mutation 1 */
const res1 = await createProduct(data);
// Kiểm tra thủ công
if (res1.data) {
    const id = res1.data.id;
    /* Mutation 2 */
    const res2 = await uploadImage({ id, file });
    // Kiểm tra thủ công
    if (res2.data) {
        toast.success("Xong hết!");
    } else {
        toast.error("Tạo được nhưng upload ảnh xịt!");
    }
} else {
    toast.error("Tạo sản phẩm đã xịt rồi!");
}

In [ ]:
// Có unwrap() (Thẳng tắp và đẹp):

try {
    const product = await createProduct(data).unwrap(); // Bước 1
    await uploadImage({ id: product.id, file }).unwrap(); // Bước 2
    
    toast.success("Xong hết!"); // Chỉ hiện khi cả 2 bước đều ngon
} catch (error) {
    toast.error("Có lỗi xảy ra ở bước nào đó!"); // Bắt lỗi của cả bước 1 hoặc bước 2
}

2. Tư duy Code: Nó viết lại Promise như thế nào?
- Nếu chúng ta tự viết lại hàm `.unwrap()` bằng JavaScript thuần, logic của nó sẽ trông nôm na như thế này:

In [ ]:
// Giả lập hàm unwrap hoạt động ngầm bên trong
const unwrap = (resultFromRedux) => {
    return new Promise((resolve, reject) => {
        
        // 1. Kiểm tra xem kết quả có lỗi không?
        if (resultFromRedux.error) {
            // NẾU CÓ LỖI:
            // Thay vì trả về object lỗi nhẹ nhàng, nó "kích nổ" (reject) luôn
            // Điều này khiến code nhảy thẳng vào block catch()
            reject(resultFromRedux.error); 
        } 
        
        // 2. Nếu không lỗi (Thành công)
        else {
            // NẾU THÀNH CÔNG:
            // Nó vứt bỏ cái vỏ bọc, chỉ trả về đúng cái ruột (data)
            resolve(resultFromRedux.data);
        }
    });
};

In [ ]:
3. Tại sao RTK không làm thế này ngay từ đầu?
? Thắc mắc: "Sao không để mặc định như vậy cho tiện":
* Lý do là Triết lý của Redux: Redux không thích làm ứng dụng bị "crash" (sập) hoặc gián đoạn luồng dữ liệu chỉ vì một API bị lỗi. 
  - Mặc định Redux muốn lưu trữ cả lỗi vào Store (`state.error`) để hiển thị lên UI một cách bị động (ví dụ: hiện dòng chữ đỏ trên màn hình).
  - Nhưng khi bạn cần xử lý chủ động (ví dụ: bấm nút -> chờ -> chuyển trang), thì cách "an toàn" của Redux lại trở thành rào cản. Đó là lúc `.unwrap()` xuất hiện như một công cụ chuyển đổi chế độ.

# Tóm lại
  - Không `unwrap`: Nhận hộp -> Mở hộp -> Kiểm tra hàng -> Dùng
  - Có `unwrap`: Nhận hàng xịn (hoặc nghe báo động lỗi)